<a href="https://colab.research.google.com/github/Chaa00/Reverse-Hex-game-/blob/main/RHEX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementation du jeu RHEX.

Projet Recherche Monte Carlo 

M2 IASD Tunis Dauphine

Fait par Mhalla Chadha et Ouled El Gourya Yesmine

# **RHEX**

Reverse hex est un jeu qui se joue sur un plateau en forme de losange composé de cellules hexagonales. Deux joueurs placent alternativement des pierres blanches et noires sur les cellules. Comme dans le jeu de l'hexagone, deux côtés non adjacents du plateau sont attribués au joueur noir, et les deux autres côtés sont attribués au joueur blanc, mais la condition de victoire est à l'opposé de celle de l'hexagone. Le premier joueur qui relie ses côtés perd.


Ci dessous une implémentation du jeu  "Reverse Hex" avec un ensemble de joueurs. Il comprend un joueur de base basé sur la recherche arborescente de Monte Carlo avec une estimation de la valeur de l'action rapide (RAVE) modifiée à partir de mopyhex et quatre variations explorant les améliorations possibles du joueur. Tous les joueurs autres que le joueur de base utilisent un livre d'ouverture et une stratégie statique de victoire garantie pour les tableaux plus petits que 6 x 6 pour le joueur ayant l'avantage de l'ordre du tour. Les classes des joueurs sont :


**rave_mctsagent:** MCTS basique avec RAVE player 

**dca_mctsagent:** celle qui utilise l'analyse des cellules mortes 

**poolrave_mctsagent:** celle qui utilise la variante poolRAVE de la MCTS

**lgr_mctsagent:** celui qui utilise l'algorithme de la dernière bonne réponse

**decisive_move_mctsagent:**  celle qui utilise les mouvements décisifs dans le the tree search rollout policy


In [2]:
class Unionfind:
  """
  Unionfind data structure spécialisée dans la recherche de Hex Connexion 
  Implementation inspirée par UAlberta CMPUT 275 2015 class notes.
  """
  def __init__(self):
    """
Initialisez le parent et classez-le comme dictionnaire vide, nous ajouterons paresseusement des éléments si nécessaire.
    """
    self.parent = {}
    self.rank = {}
    self.groups = {}
    self.ignored = []

  def join(self, x, y):
    """
    fusionner les groupes de x et y si ils n'ont pas été deja fusionnés  ,
    return False si ils sont déja été fusionnés , true sinon
    """
    rep_x = self.find(x)
    rep_y = self.find(y)
    if rep_x == rep_y:
      return False
    if self.rank[rep_x] < self.rank[rep_y]:
      self.parent[rep_x] = rep_y
      self.groups[rep_y].extend(self.groups[rep_x])
      del self.groups[rep_x]
    elif self.rank[rep_x] >self.rank[rep_y]:
      self.parent[rep_y] = rep_x
      self.groups[rep_x].extend(self.groups[rep_y])
      del self.groups[rep_y]      
    else:
      self.parent[rep_x] = rep_y
      self.rank[rep_y] += 1
      self.groups[rep_y].extend(self.groups[rep_x])
      del self.groups[rep_x]
    return True

  def find(self, x):
    """
Obtenez l'élément représentatif associé à l'ensemble dans lequel réside l'élément x. Utilise la compression grand-parent pour la compression
l'arborescence à chaque opération de recherche afin que les futures opérations de recherche soient plus rapides.
    """
    if x not in self.parent:
      self.parent[x] = x
      self.rank[x] = 0
      if x in self.ignored:
        self.groups[x] = []
      else:
        self.groups[x] = [x]
    px = self.parent[x]
    if x == px: return x
    gx = self.parent[px]
    if gx==px: return px

    self.parent[x] = gx

    return self.find(gx)

  def connected(self, x, y):
    """vérifier si deux éléments sont de meme groupe."""
    return self.find(x)==self.find(y)
  
  def set_ignored_elements(self, ignore):
    """Définir les éléments dans ignoré."""
    self.ignored = ignore
  
  def get_groups(self):
    """Renvoyer le dictionnaire des groupes de cellules connectées."""
    return self.groups

In [3]:
import numpy as np
from collections import deque
class Gamestate:
  """
Stocke des informations représentant l'état actuel d'un jeu d'hexagone, à savoir la planche et le tour actuel. Fournit également des fonctions pour jouer au jeu
et renvoyer des informations à ce sujet.
  """
  # dictionnaire associant des nombres à des joueurs pour la comptabilité
  PLAYERS = {"none" : 0, "white" : 1, "black" : 2}
  OPPONENT ={0 : 0, 1 : 2, 2 : 1}
  DARK_STATE = {"unknown" : 0, "visible" : 1}

  # move value of -1 indique que le jeu est terminé, donc aucun mouvement n'est possible
  GAMEOVER = -1 
  #représenter les arêtes (edges) dans l'union find structure pour la détection de victoire
  EDGE1 = 1
  EDGE2 = 2

  DISP_WHITE = 'O'
  DISP_BLACK = '@'
  DISP_EMPTY = '.'
  neighbor_patterns = ((-1,0), (0,-1), (-1,1), (0,1), (1,0), (1,-1))

  def __init__(self, size):
    """
    Initialiser le jeu et donner le remier tour aux blancs
    et créer notre union find structures pour la vérification des gagnants.
    """
    self.size = size
    self.toplay = self.PLAYERS["white"]
    self.board = np.zeros((size, size))
    self.revealed = np.zeros((size, size))
    self.white_groups = Unionfind()
    self.black_groups = Unionfind()
    self.white_groups.set_ignored_elements([self.EDGE1, self.EDGE2])
    self.black_groups.set_ignored_elements([self.EDGE1, self.EDGE2])
    self.num_played = 0
    self.empty = set()
    for i in range(self.size):
      for j in range(self.size):
        self.empty.add((i, j))
        
  def get_white_groups(self):
    return self.white_groups.get_groups()
  
  def get_black_groups(self):
    return self.black_groups.get_groups()

  def play(self, cell):
    """
Jouer une pierre de la couleur des tours actuels dans la cellule passée.
    """
    if(self.toplay == self.PLAYERS["white"]):
      self.place_white(cell)
      self.toplay = self.PLAYERS["black"]
    elif(self.toplay == self.PLAYERS["black"]):
      self.place_black(cell)
      self.toplay = self.PLAYERS["white"]
    self.num_played += 1

  def place_white(self, cell):
    """
Placer la pierre blanche quel que soit son tour.
    """
    if(self.board[cell] == self.PLAYERS["none"]):
      self.board[cell] = self.PLAYERS["white"]
      self.empty.remove(cell)
    else:
      raise ValueError("Cell occupied")
    # si la cellule placée touche un bord blanc, connectez-la correctement
    if(cell[0] == 0):
      self.white_groups.join(self.EDGE1, cell)
    if(cell[0] == self.size - 1):
      self.white_groups.join(self.EDGE2, cell)
    #rejoindre tous les groupes connectés par la nouvelle pierre blanche
    for n in self.neighbors(cell):
      if(self.board[n] == self.PLAYERS["white"]):
        self.white_groups.join(n, cell)

  def place_black(self, cell):
    """
Placer une pierre noire quel que soit son tour.
    """
    if(self.board[cell] == self.PLAYERS["none"]):
      self.board[cell] = self.PLAYERS["black"]
      self.empty.remove(cell)
    else:
      raise ValueError("Cell occupied")
    #si la cellule placée touche un bord noir, connectez-la correctement
    if(cell[1] == 0):
      self.black_groups.join(self.EDGE1, cell)
    if(cell[1] == self.size - 1):
      self.black_groups.join(self.EDGE2, cell)
    #rejoindre tous les groupes connectés par la nouvelle pierre noire
    for n in self.neighbors(cell):
      if(self.board[n] == self.PLAYERS["black"]):
        self.black_groups.join(n, cell)

  def turn(self):
    """
    Return le joueur avec le coup suivant.
    """
    return self.toplay

  def set_turn(self, player):
    """
    Configurer le joueur pour qu'il effectue le prochain mouvement.
    """
    if(player in self.PLAYERS.values() and player != self.PLAYERS["none"]):
      self.toplay = player
    else:
      raise ValueError('Invalid turn: ' + str(player))

  def winner(self):
    """
     Renvoyer un numéro correspondant au joueur gagnant,ou aucun si le jeu n'est pas terminé.
    """
    if(self.white_groups.connected(self.EDGE1, self.EDGE2)):
      return self.PLAYERS["black"]
    elif(self.black_groups.connected(self.EDGE1, self.EDGE2)):
      return self.PLAYERS["white"]
    else:
      return self.PLAYERS["none"]
    
  def would_lose(self, cell, color):
    """
    Return True est le coup indiqué par la cellule et la couleur perdrait la partie, False sinon 
    """
    connect1 = False
    connect2 = False
    if color == self.PLAYERS["black"]:
      if cell[1] == 0:
        connect1 = True
      elif cell[1] == self.size - 1:
        connect2 = True
      for n in self.neighbors(cell):
        if self.black_groups.connected(self.EDGE1, n):
          connect1 = True
        elif self.black_groups.connected(self.EDGE2, n):
          connect2 = True
    elif color == self.PLAYERS["white"]:
      if cell[0] == 0:
        connect1 = True
      elif cell[0] == self.size - 1:
        connect2 = True
      for n in self.neighbors(cell):
        if self.white_groups.connected(self.EDGE1, n):
          connect1 = True
        elif self.white_groups.connected(self.EDGE2, n):
          connect2 = True
      
    return connect1 and connect2
  
  def show_board(self, player):
    ret = '\n'
    coord_size = len(str(self.size))
    offset = 1
    ret += ' ' * (offset + 1)
    for x in range(self.size):
      ret+=chr(ord('A') + x) +' ' * offset * 2
    ret += '\n'
    for y in range(self.size):
      ret += str(y + 1) + ' ' * (offset * 2 + coord_size - len(str(y + 1)))
      for x in range(self.size):
        if(self.board[x, y] == self.PLAYERS["white"] and
           (self.revealed[x, y] == self.DARK_STATE["visible"] or
            player == self.PLAYERS["white"])):
          ret += self.DISP_WHITE
        elif(self.board[x,y] == self.PLAYERS["black"] and
             (self.revealed[x, y] == self.DARK_STATE["visible"] or
              player == self.PLAYERS["black"])):
          ret += self.DISP_BLACK
        else:
          ret += self.DISP_EMPTY
        ret += ' ' * offset * 2
      ret += self.DISP_WHITE + "\n" + ' ' * offset * (y + 1)
    ret += ' ' * (offset * 2 + 1) 
    ret += (self.DISP_BLACK + ' ' * offset * 2) * self.size

    return ret      

  def neighbors(self, cell, color=None):
    """Renvoie la liste des voisins de la cellule passée."""
    if cell == self.EDGE1:
      if color == self.PLAYERS["black"]:
        nb = []
        for i in range(self.size):
          nb.append((i, 0))
      elif color == self.PLAYERS["white"]:
        nb = []
        for i in range(self.size):
          nb.append((0, i))
      return nb
    elif cell == self.EDGE2:
      nb = []
      if color == self.PLAYERS["black"]:
        for i in range(self.size):
          nb.append((i, self.size -1))
      elif color == self.PLAYERS["white"]:
        for i in range(self.size):
          nb.append((self.size -1, i))
      return nb
    x = cell[0]
    y=cell[1]
    return [(n[0]+x , n[1]+y) for n in self.neighbor_patterns\
      if (0<=n[0]+x and n[0]+x<self.size and 0<=n[1]+y and n[1]+y<self.size)]

  def moves(self):
    """ obtenir une liste de tous les mouvements possibles sur le plateau actuel ."""
    moves = []
    for y in range(self.size):
      for x in range(self.size):
        if self.board[x,y] == self.PLAYERS["none"]:
          moves.append((x,y))
    return moves
  
  def blank_ldiagonal(self):
    """
    Returns True si toutes les cellules sur la longue diagonale du tableau sont vides. False sinon.
    """
    for i in range(self.size):
      if self.board[i, i] != self.PLAYERS["none"]:
        return False
    return True
    
  def blank_sdiagonal(self):
    """
    Returns True si toutes les cellules de la diagonale courte du tableau sont vides. False sinon.
    """
    for i in range(self.size):
      if self.board[i, self.size - i - 1] != self.PLAYERS["none"]:
        return False
    return True
  
  def get_color(self, cell):
    """Returns la couleur de la cellule."""
    return self.board[cell[0], cell[1]]
  
  def reachable(self, colors, stopset, start):
    """
Retourne un ensemble contenant les cellules atteignables à partir de start en passant par les cellules d'une couleur dans colors et sans croiser les cellules dans stopet.
    """
    seen = set([start])
    queue = deque([start])
    while len(queue) > 0:
      cell = queue.popleft()
      if cell in stopset:
        continue
      
      for nb in self.neighbors(cell, colors[0]):
        if self.get_color(nb) in colors and nb not in seen:
          queue.append(nb)
          seen.add(nb)
          
    return seen
  
  def get_empty_cell_set(self):
    """ Retourne l'ensemble des cellules vides."""
    return self.empty
  
  def connected(self, color, cell1, cell2):
    if color == self.PLAYERS["black"]:
      return self.black_groups.connected(cell1, cell2)
    elif color == self.PLAYERS["white"]:
      return self.white_groups.connected(cell1, cell2)

  def __str__(self):
    """ecrire une représentation ascii du plateau de jeu."""
    ret = '\n'
    coord_size = len(str(self.size))
    offset = 1
    ret+=' '*(offset+1)
    for x in range(self.size):
      ret+=chr(ord('A')+x)+' '*offset*2
    
    ret+='\n'
    for y in range(self.size):
      ret+=str(y+1)+' '*(offset*2+coord_size-len(str(y+1)))
      for x in range(self.size):
        if(self.board[x, y] == self.PLAYERS["white"]):
          ret+=self.DISP_WHITE
        elif(self.board[x,y] == self.PLAYERS["black"]):
          ret+=self.DISP_BLACK
        else:
          ret+=self.DISP_EMPTY
        ret+=' '*offset*2
      ret+=self.DISP_WHITE+"\n"+' '*offset*(y+1)
    ret+=' '*(offset*2+1)+(self.DISP_BLACK+' '*offset*2)*self.size

    return ret

In [4]:
import time
from math import sqrt, log
import random
from copy import copy, deepcopy
from sys import stderr
from queue import Queue
inf = float('inf')

class Node:
  """
  Node pour le MCST. Stocke le mouvement appliqué pour atteindre ce noeud à partir de son parent, 
  les states pour la position du jeu associée, les enfants, le parent et le outcome. 
  (outcome==none à moins que la position ne mette fin au jeu).
  """
  
  def __init__(self, move = None, parent = None):
    """
    Initialiser un nouveau noeud avec déplacement facultatif et parent et liste d'enfants initialement vide.
    un rollout statistics et un outcome non spécifié
    """
    self.move = move
    self.parent = parent
    self.N = 0 #nombre de fois que cette position a été visitée
    self.Q = 0 #le rendement moyen (gains/pertes) de cette position
    self.children = []
    self.outcome = Gamestate.PLAYERS["none"]

  def add_children(self, children):
    """ Ajouter une liste des noeuds aux children de ce noeud    """
    self.children += children

  def set_outcome(self, outcome):
    """ Définir le outcome de ce noeud , càd si nous décidons que le noeud est la fin du jeu
    """
    self.outcome = outcome

  def value(self, explore):
    """
Calculer la valeur UCT de ce nœud par rapport à son parent, 
le paramètre "explore" spécifie dans quelle mesure la valeur doit favoriser les nœuds qui n'ont pas encore été explorés en profondeur par rapport aux noeuds
qui semblent avoir un taux de victoire élevé. 
Actuellement, explore est mis à zéro lors du choix du meilleur coup à jouer afin que le coup avec le plus haut taux de victoire soit toujours choisi. 
"""
    #sauf si explore ne soit fixé à zéro, favorise au maximum les noeuds non explorés
    if(self.N == 0):
      if(explore == 0):
        return 0
      else:
        return inf
    else:
      return self.Q/self.N + explore*sqrt(2*log(self.parent.N)/self.N)


class Mctsagent:
  """ Implémentation d'un agent qui effectue la MCTS pour Hex """
  EXPLORATION = 1  

  def __init__(self, state=Gamestate(8)):
    self.rootstate = deepcopy(state)
    self.root = Node()

  def best_move(self):
    """ Retourne le meilleur déplacement selon l'arbre actuel.
    """
    if(self.rootstate.winner() != Gamestate.PLAYERS["none"]):
      return Gamestate.GAMEOVER

    #choisir le déplacement du noeud le plus simulé en brisant les égalités de façon aléatoire
    max_value = max(self.root.children, key = lambda n: n.N).N
    max_nodes = [n for n in self.root.children if n.N == max_value]
    bestchild = random.choice(max_nodes)
    return bestchild.move

  def move(self, move):
    """Faites le pas passé et mettez l'arbre à jour de manière appropriée."""
    for child in self.root.children:
      #faire de l'enfant associé au déplacement la nouvelle racine
      if move == child.move:
        child.parent = None
        self.root = child
        self.rootstate.play(child.move)
        return

    #si, pour une raison quelconque, le déplacement n'est pas dans les enfants de la racine, il suffit de jeter l'arbre et de recommencer.
    self.rootstate.play(move)
    self.root = Node()

  def search(self, time_budget):
    """Recherche et mise à jour de l'arbre de recherche pendant une durée spécifiée en secondes."""
    startTime = time.clock()
    num_rollouts = 0
    #faire jusqu'à ce que nous dépassions notre budget de temps
    while(time.clock() - startTime <time_budget):
      node, state = self.select_node()
      turn = state.turn()
      outcome = self.roll_out(state)
      self.backup(node, turn, outcome)
      num_rollouts += 1
    stderr.write("Ran "+str(num_rollouts)+ " rollouts in " +\
            str(time.clock() - startTime)+" sec\n")
    stderr.write("Node count: "+str(self.tree_size())+"\n")
  def select_node(self):
    """Sélectionnez un nœud dans l'arbre pour effectuer une seule simulation."""
    node = self.root
    state = deepcopy(self.rootstate)

    #s'arrêter si nous trouvons un " leaf node"
    while(len(node.children) !=0 ):
      #descendre jusqu'au noeud de valeur maximale, rompre les liens au hasard
      max_node = max(node.children, key = lambda n: n.value(self.EXPLORATION))
      max_value = max_node.value(self.EXPLORATION)
      max_nodes = [n for n in node.children if n.value(self.EXPLORATION) == max_value]
      node = random.choice(max_nodes)
      state.play(node.move)

      #si un noeud enfant n'a pas été exploré, sélectionnez le avant de développer les autres enfants.
      if node.N == 0:
        return (node, state)

    #si on atteint un noeud feuille, on génère ses enfants et on retourne l'un d'entre eux.
    #si le noeud est terminal, retourne simplement le noeud terminal
    if(self.expand(node, state)):
      node = random.choice(node.children)
      state.play(node.move)
    return (node, state)

  def roll_out(self, state):
    """Simule un jeu entièrement aléatoire à partir de l'état transmis et renvoie le joueur gagnant."""
    moves = state.moves()
    while(state.winner() == Gamestate.PLAYERS["none"]):
      move = random.choice(moves)
      state.play(move)
      moves.remove(move)
    return state.winner()

  def backup(self, node, turn, outcome):
    """Mettre à jour les statistiques des noeuds sur le chemin allant du nœud passé à la racine pour refléter le résultat d'un jeu simulé de façon aléatoire.."""
    #Notez que la récompense est calculée pour le joueur qui vient de jouer au noeud et non au prochain joueur à jouer
    reward = -1 if outcome == turn else 1

    while node != None:
      node.N += 1
      node.Q +=reward
      reward = -reward
      node = node.parent

  def expand(self, parent, state):
    """ Génère les enfants du noeud "parent" transmis en fonction des mouvements disponibles dans l'état de jeu transmis et les ajoute à l'arbre."""
    children = []
    if(state.winner() != Gamestate.PLAYERS["none"]):
    #le jeu est terminé à ce nœud donc rien à développer
      return False
    for move in state.moves():
      children.append(Node(move, parent))
    parent.add_children(children)
    return True

  def set_gamestate(self, state):
    """ Définir l'état de la racine de l'arbre à game_state passé,ceci efface toutes les informations stockées dans l'arbre puisqu'elles ne s'appliquent pas au nouvel état.."""
    self.rootstate = deepcopy(state)
    self.root = Node()

  def tree_size(self):
    """Compter les  noeuds dans l'arbre par BFS."""
    Q = Queue()
    count = 0
    Q.put(self.root)
    while not Q.empty():
      node = Q.get()
      count +=1
      for child in node.children:
        Q.put(child)
    return count

In [5]:
class Rave_Node(Node):
  
  def __init__(self, move = None, parent = None):
    """Initialise un nouveau noeud avec les options move et parent et une liste d'enfants initialement vide, ainsi que des statistiques de déploiement et un résultat non spécifié.."""
    self.move = move
    self.parent = parent
    self.N = 0 
    self.Q = 0 
    self.Q_RAVE = 0 # nombre de fois ce mouvement s'est avéré décisif dans un rollout
    self.N_RAVE = 0 # nombre de fois que ce mouvement est apparu dans un rollout
    self.children = {}
    self.outcome = Gamestate.PLAYERS["none"]

  def add_children(self, children):
    for child in children:
      self.children[child.move] = child

  def value(self, explore, crit):

    #à moins que explore ne soit fixé à zéro, favorise au maximum les noeuds non explorés
    if(self.N == 0):
      if(explore == 0):
        return 0
      else:
        return inf
    else:
      #rave valuation:
      alpha = max(0,(crit - self.N)/crit)
      return self.Q*(1-alpha)/self.N+self.Q_RAVE*alpha/self.N_RAVE


class RaveMctsagent(Mctsagent):
  RAVE_CONSTANT = 300
  EXPLORATION = 1
  
  CASES = {}
  CASE_FIRST = {}
  CASE_SINGLE = {}

  CASES[2] = {(0, 1): (1, 1), (1, 1): (0, 1)}
  CASE_FIRST[2] = [(0, 0)]
  CASE_SINGLE[2] = (1, 0)

  CASES[3] = {(0, 1): (0, 2), (0, 2): (0, 1), (2, 0): (2, 1), (2, 1): (2, 0)}
  CASE_FIRST[3] = [(0, 0), (1, 0), (1, 2), (2, 2)]
  CASE_SINGLE[3] = (1, 1)

  CASES[4] = {(2, 0): (3, 0), (3, 0): (2, 0), (2, 2): (3, 1), (3, 1): (2, 2),
              (1, 2): (3, 2), (3, 2): (1, 2), (0, 3): (1, 3), (1, 3): (0, 3),
              (2, 3): (3, 3), (3, 3): (2, 3)}
  CASE_FIRST[4] = [(0, 0), (1, 0), (0, 1), (0, 2), (1, 1)]
  CASE_SINGLE[4] = (2, 1)

  CASES[5] = {(0, 1): (0, 2), (0, 2): (0, 1), (0, 3): (0, 4), (0, 4): (0, 3),
              (1, 1): (1, 3), (1, 3): (1, 1), (1, 2): (2, 1), (2, 1): (1, 2),
              (3, 1): (3, 3), (3, 3): (3, 1), (2, 3): (3, 2), (3, 2): (2, 3),
              (4, 0): (4, 1), (4, 1): (4, 0), (4, 2): (4, 3),(4, 3): (4, 2)}
  CASE_FIRST[5] = [(0, 0), (1, 0), (2, 0), (3, 0), (1, 4), (2, 4), (3, 4),
                   (4, 4)]
  CASE_SINGLE[5] = (2, 2)  
  
  def __init__(self, state=Gamestate(8)):
    self.set_gamestate(state)
    self.short_symetrical = True
  
  def special_case(self, last_move):
    """Retourne un move trouvé sans recherche, Aucun sinon."""
    return None

  def best_move(self):
    """Return le meilleur move en fonction de l'arbre actuel."""
    if(self.rootstate.winner() != Gamestate.PLAYERS["none"]):
      return gamestate.GAMEOVER

    #choisir le déplacement du noeud le plus simulé en brisant les égalités de façon aléatoire
    max_value = max(self.root.children.values(), key = lambda n: n.N).N
    max_nodes = [n for n in self.root.children.values() if n.N == max_value]
    bestchild = random.choice(max_nodes)
    return bestchild.move

  def move(self, move):
    """Faire le passed move et mettre l'arbre à jour d une manière appropriée.."""
    if move in self.root.children:
      child = self.root.children[move]
      child.parent = None
      self.root = child
      self.rootstate.play(child.move)
      return
    #si pour une raison quelconque, le déplacement n'est pas dans les enfants de la racine, il suffit de jeter l'arbre et de recommencer.
    self.rootstate.play(move)
    self.root = Rave_Node()

  def search(self, time_budget):
    """Recherche et mise à jour de l'arbre de recherche pendant une durée spécifiée en secondes."""
    startTime = time.clock()
    num_rollouts = 0
    while(time.clock() - startTime < time_budget):
      node, state = self.select_node()
      turn = state.turn()
      outcome, black_rave_pts, white_rave_pts = self.roll_out(state)
      self.backup(node, turn, outcome, black_rave_pts, white_rave_pts)
      num_rollouts += 1

  def select_node(self):
    node = self.root
    state = deepcopy(self.rootstate)
    #s'arrêter si nous atteignons un nœud feuille
    while(len(node.children) != 0):
      max_node = max(node.children.values(),key=lambda n: n.value(self.EXPLORATION, self.RAVE_CONSTANT))
      max_value = max_node.value(self.EXPLORATION, self.RAVE_CONSTANT)
      #decendre à la valeur maximale du noeud
      max_nodes = [n for n in node.children.values() if n.value(self.EXPLORATION, self.RAVE_CONSTANT) == max_value]
      node = random.choice(max_nodes)
      state.play(node.move)
#si un nœud enfant n'a pas été exploré, sélectionnez-le avant de développer les autres enfants.
      if node.N == 0:
        return (node, state)
    if(self.expand(node, state)):
      node = random.choice(list(node.children.values()))
      state.play(node.move)
    return (node, state)

  def backup(self, node, turn, outcome, black_rave_pts, white_rave_pts):
    """Mettre à jour les statistiques des neuds sur le chemin du npeud passé à la racine pour refléter le résultat d'une simulation aléatoire"""
    # notez que la récompense est calculée pour le joueur qui vient de jouer au nœud et non pour le prochain joueur à jouer
    reward = -1 if outcome == turn else 1
    while node != None:
      if turn == Gamestate.PLAYERS["white"]:
        for point in white_rave_pts:
          if point in node.children:
            node.children[point].Q_RAVE += -reward
            node.children[point].N_RAVE += 1
      else:
        for point in black_rave_pts:
          if point in node.children:
            node.children[point].Q_RAVE += -reward
            node.children[point].N_RAVE += 1

      node.N += 1
      node.Q += reward
      if turn == Gamestate.PLAYERS["black"]:
        turn = Gamestate.PLAYERS["white"]
      else:
        turn = Gamestate.PLAYERS["black"]
      reward = -reward
      node = node.parent

  def expand(self, parent, state):
    """ Génère les enfants du nœud "parent" transmis en fonction des mouvements disponibles dans l'état de jeu transmis et les ajoute à l'arbre."""
    children = []
    if(state.winner() != Gamestate.PLAYERS["none"]): 
      return False
    for move in state.moves():
      children.append(Rave_Node(move, parent))
    parent.add_children(children)
    return True

  def set_gamestate(self, state):
    """Définir l'état de la racine de l'arbre au gamestate passé, ceci efface toutes les informations stockées dans l'arbre puisqu'elles ne s'appliquent pas au nouvel état."""
    self.rootstate = deepcopy(state)
    self.root = Rave_Node()

  def roll_out(self, state):
    """Simuler un jeu aléatoire, sauf que nous jouons d'abord toutes les cellules critiques connues, 
       puis nous retournons le joueur gagnant et enregistrons les cellules critiques à la fin. """
    moves = state.moves()
    while(state.winner() == Gamestate.PLAYERS["none"]):
      move = random.choice(moves)
      state.play(move)
      moves.remove(move)
    black_rave_pts = []
    white_rave_pts = []
    for x in range(state.size):
      for y in range(state.size):
        if state.board[(x,y)] == Gamestate.PLAYERS["black"]:
          black_rave_pts.append((x,y))
        elif state.board[(x,y)] == Gamestate.PLAYERS["white"]:
          white_rave_pts.append((x,y))
    return state.winner(), black_rave_pts, white_rave_pts
  def tree_size(self):
    Q = Queue()
    count = 0
    Q.put(self.root)
    while not Q.empty():
      node = Q.get()
      count += 1
      for child in node.children.values():
        Q.put(child)
    return count  
  
  def get_small_board_move(self, last_move, size, moves):
    """Retourner les coups de cas spéciaux sur les plateaux de 5x5 ou plus petits Si aucun coup de ce type ne peut être trouvé, retourner None."""
    turn = self.rootstate.turn()
    if not((size % 2 == 0 and turn == Gamestate.PLAYERS["white"]) or 
           (size % 2 == 1 and turn == Gamestate.PLAYERS["black"])):
      return None
    case = self.CASES[size]
    if last_move in case and case[last_move] in moves:
      return case[last_move]
    elif (last_move in case and case[last_move] not in moves or 
          last_move == self.CASE_SINGLE[size]):
      for key in case.keys():
        if key in moves and case[key] in moves:
          return key
    for cell in self.CASE_FIRST[size]:
      if cell in moves:
        return cell    

  def get_starting_move(self, last_move, size, moves):
    """Retourne les cas spéciaux pour les mouvements pour le début de la partie. Si aucun coup ne peut être trouvé, retourne None."""
    far_corner = (size - 1, size - 1)
    if self.rootstate.num_played == 0 and size % 2 == 0:
      # Jouez d'abord dans les coins aigus.
      return random.choice([(0, 0), far_corner])
    elif last_move == None:
      return None
    short_diagonal_reflect = (size - 1 - last_move[0], size - 1 - last_move[1])
    if self.rootstate.num_played == 1 and size % 2 == 1:
      # si le premier move n'était pas sur la longue diagonale
      if last_move[0] != last_move[1]:
        # Jouer le reflet de ce move dans la longue diagonale
        return (last_move[1], last_move[0])
      # If premier move était dans le centre
      elif last_move[0] == (size - 1) / 2 and last_move[1] == (size - 1) / 2:
        return random.choice((0, 0), far_corner)
      else:
        # Jouez la réflexion en diagonale courte
        return short_diagonal_reflect
    # Maintenez la symétrie avec les mouvements de l'adversaire aussi longtemps que possible.
    elif size % 2 == 1 and self.rootstate.num_played % 2 == 1:
      if self.rootstate.blank_ldiagonal():
        self.short_symetrical = False
        return (last_move[1], last_move[0])
      
      elif (self.rootstate.blank_sdiagonal() 
          and self.short_symetrical):
        return short_diagonal_reflect
      
    return None

In [6]:
class PoolraveMctsagent(RaveMctsagent):
  
  def __init__(self, state=Gamestate(8)):
    super().__init__(state)
    self.black_rave = {}
    self.white_rave = {} 
  
  def special_case(self, last_move):
    """Retourne un move trouvé sans recherche, aucun sinon ."""
    size = self.rootstate.size
    moves = self.rootstate.moves()
    
    if size < 6:
      move = self.get_small_board_move(last_move, size, moves)
      if move is not None:
        return move
    
    move = self.get_starting_move(last_move, size, moves)
    return move    

  def roll_out(self, state):
    """Simuler un jeu aléatoire, sauf que nous jouons d'abord toutes les cellules critiques connues, 
    puis nous retournons le joueur gagnant et enregistrons les cellules critiques à la fin.
    """
    moves = state.moves()
    black_rave_moves = sorted(self.black_rave.keys(),
                              key=lambda cell: self.black_rave[cell])
    white_rave_moves = sorted(self.white_rave.keys(),
                              key=lambda cell: self.white_rave[cell])
    black_pool = []
    white_pool = []
    i = 0
    while len(black_pool) < 10 and i < len(black_rave_moves):
      if black_rave_moves[i] in moves:
        black_pool.append(black_rave_moves[i])
      i += 1
      
    i = 0
    while len(white_pool) < 10 and i < len(white_rave_moves):
      if white_rave_moves[i] in moves:
        white_pool.append(white_rave_moves[i])
      i += 1
    num_pool = 0
    while(state.winner() == Gamestate.PLAYERS["none"]):
      move = None
      if len(black_pool) > 0 and state.turn() == Gamestate.PLAYERS["black"]:
        move = random.choice(black_pool)
        num_pool += 1
      elif len(white_pool) > 0:
        move = random.choice(white_pool)
        num_pool += 1
      if random.random() > 0.5 or not move or move not in moves:
        move = random.choice(moves)
        num_pool -= 1
      state.play(move)
      moves.remove(move)

    black_rave_pts = []
    white_rave_pts = []

    for x in range(state.size):
      for y in range(state.size):
        if state.board[(x, y)] == Gamestate.PLAYERS["black"]:
          black_rave_pts.append((x, y))
          if state.winner() == Gamestate.PLAYERS["black"]:
            if (x, y) in self.black_rave:
              self.black_rave[(x, y)] += 1
            else:
              self.black_rave[(x, y)] = 1
          else:
            if (x, y) in self.black_rave:
              self.black_rave[(x, y)] -= 1
            else:
              self.black_rave[(x, y)] = -1
        elif state.board[(x, y)] == Gamestate.PLAYERS["white"]:
          white_rave_pts.append((x, y))
          if state.winner() == Gamestate.PLAYERS["white"]:
            if (x, y) in self.white_rave:
              self.white_rave[(x, y)] += 1
            else:
              self.white_rave[(x, y)] = 1
          else:
            if (x, y) in self.white_rave:
              self.white_rave[(x, y)] -= 1
            else:
              self.white_rave[(x, y)] = -1

    return state.winner(), black_rave_pts, white_rave_pts

In [8]:
class DecisiveMoveMctsagent(RaveMctsagent):
  
  def __init__(self, state=Gamestate(8)):
    super().__init__(state)
  
  def special_case(self, last_move):
    """Return a move found without search, None otherwise."""
    size = self.rootstate.size
    moves = self.rootstate.moves()
    
    if size < 6:
      move = self.get_small_board_move(last_move, size, moves)
      if move is not None:
        return move
    
    move = self.get_starting_move(last_move, size, moves)
    return move

  def roll_out(self, state):
    """Simuler un jeu aléatoire, sauf que nous jouons d'abord toutes les cellules critiques connues, renvoyons le joueur gagnant et enregistrons les cellules critiques à la fin."""
    moves = state.moves()
    good_moves = moves.copy()
    good_opponent_moves = moves.copy()
    to_play = state.turn()
    while(state.winner() == Gamestate.PLAYERS["none"]):
      done = False
      while len(good_moves) > 0 and not done:
        move = random.choice(good_moves)
        good_moves.remove(move)
        if not state.would_lose(move, to_play):
          state.play(move)
          moves.remove(move)
          if move in good_opponent_moves:
            good_opponent_moves.remove(move)
          done = True
      
      if not done:    
        move = random.choice(moves)
        state.play(move)
        moves.remove(move)
        if move in good_opponent_moves:
          good_opponent_moves.remove(move)
          
      good_moves, good_opponent_moves = good_opponent_moves, good_moves
    
    black_rave_pts = []
    white_rave_pts = []

    for x in range(state.size):
      for y in range(state.size):
        if state.board[(x,y)] == Gamestate.PLAYERS["black"]:
          black_rave_pts.append((x,y))
        elif state.board[(x,y)] == Gamestate.PLAYERS["white"]:
          white_rave_pts.append((x,y))

    return state.winner(), black_rave_pts, white_rave_pts

In [9]:
class DCAMctsagent(RaveMctsagent):
  
  RAVE_CONSTANT = 300

  def __init__(self, state=Gamestate(8)):
    super().__init__(state)
    self.dead = set()

  def special_case(self, last_move):
    """Return a move found without search, None otherwise."""
    size = self.rootstate.size
    moves = self.rootstate.moves()
      
    if size < 6:
      move = self.get_small_board_move(last_move, size, moves)
      if move is not None:
        return move
    
    move = self.get_starting_move(last_move, size, moves)
    if move is not None:
      return move
      
    self.findDeadRegions()
    if len(self.dead) > 0:
      move = self.dead.pop()
      return move
    
    return None
  
  def move(self, move):
    """Faites le passed move et mettez l'arbre à jour de manière appropriée.."""
    self.dead.discard(move)
    super().move(move)
  
  def findEdgeUnreachable(self, color, stopset, checkSide1=True, checkSide2=True):
    """Trouver les zones inaccessibles d'un côté ou de l'autre du plateau en traversant uniquement les cellules vides ou de couleur et sans traverser les cellules de l'ensemble d'arrêt..
      Le calcul est ignoré pour un côté si le jeu d'arrêts le touche, indiqué par checkSide1 et checkSide2. """
    colors = [color, Gamestate.PLAYERS["none"]]
    if checkSide1:
      reachable1 = self.rootstate.reachable(colors, stopset, Gamestate.EDGE1)
    else:
      reachable1 = set()
    if checkSide2:
      reachable2 = self.rootstate.reachable(colors, stopset, Gamestate.EDGE2)
    else:
      reachable2 = set()
    
    return self.rootstate.get_empty_cell_set() - (reachable1 | reachable2)

  def findDeadRegions(self):
    """ Trouver les cellules mortes créées par un seul groupe connecté de voisins des pierres et les ajoute à self.dead."""    
    color_groups = [self.rootstate.get_black_groups(),
                    self.rootstate.get_white_groups()]
    for groups in color_groups:
      for key in groups.keys():
        group = groups[key]
        if len(group) < 2:
          continue
        color = self.rootstate.get_color(group[0])
        nb = set()     
        for cell in group:
          for neighbor in self.rootstate.neighbors(cell):
            if self.rootstate.get_color(neighbor) == Gamestate.PLAYERS["none"]:
              nb.add(neighbor)       
        checkSide1 = True
        checkSide2 = True
        if color == Gamestate.PLAYERS["black"]:
          for cell in group:
            if cell[1] == 0:
              checkSide1 = False
            elif cell[1] == self.rootstate.size - 1:
              checkSide2 = False
        elif color == Gamestate.PLAYERS["white"]:
          for cell in group:
            if cell[0] == 0:
              checkSide1 = False
            elif cell[0] == self.rootstate.size - 1:
              checkSide2 = False       
        self.dead = self.dead | self.findEdgeUnreachable(color, nb, checkSide1,checkSide2)

In [10]:
class LGRMctsagent(RaveMctsagent):
  
  def __init__(self, state=Gamestate(8)):
    super().__init__(state)
    self.black_reply = {}
    self.white_reply = {}
  
  def special_case(self, last_move):
    size = self.rootstate.size
    moves = self.rootstate.moves()
    if size < 6:
      move = self.get_small_board_move(last_move, size, moves)
      if move is not None:
        return move
    move = self.get_starting_move(last_move, size, moves)
    return move    
  def roll_out(self, state):
    moves = state.moves()
    first = state.turn()
    if first == Gamestate.PLAYERS["black"]:
      current_reply = self.black_reply
      other_reply = self.white_reply
    else:
      current_reply = self.white_reply
      other_reply = self.black_reply
    black_moves = []
    white_moves = []
    last_move = None
    while(state.winner() == Gamestate.PLAYERS["none"]):
      if last_move in current_reply:
        move = current_reply[last_move]
        if move not in moves or random.random() > 0.5:
          move = random.choice(moves)
      else:
        move = random.choice(moves)
      if state.turn() == Gamestate.PLAYERS["black"]:
        black_moves.append(move)
      else:
        white_moves.append(move)
      current_reply, other_reply = other_reply, current_reply
      state.play(move)
      moves.remove(move)
      last_move = move

    black_rave_pts = []
    white_rave_pts = []

    for x in range(state.size):
      for y in range(state.size):
        if state.board[(x,y)] == Gamestate.PLAYERS["black"]:
          black_rave_pts.append((x,y))
        elif state.board[(x,y)] == Gamestate.PLAYERS["white"]:
          white_rave_pts.append((x,y))
    
    offset = 0
    skip = 0          
    if state.winner() == Gamestate.PLAYERS["black"]:
      
      if first == Gamestate.PLAYERS["black"]:
        offset = 1
      if state.turn() == Gamestate.PLAYERS["black"]:
        skip = 1
      for i in range(len(white_moves) - skip):
        self.black_reply[white_moves[i]] = black_moves[i + offset]
    else:
      if first == Gamestate.PLAYERS["white"]:
        offset = 1
      if state.turn() == Gamestate.PLAYERS["white"]:
        skip = 1
      for i in range(len(black_moves) - skip):
        self.white_reply[black_moves[i]] = white_moves[i + offset]

    return state.winner(), black_rave_pts, white_rave_pts
  
  def set_gamestate(self, state):
    """Définir l'état de la racine de l'arbre au gamestate passé, ceci efface toutes les informations stockées dans l'arbre puisqu'elles ne s'appliquent pas au nouvel état."""
    super().set_gamestate(state)
    self.white_reply = {}
    self.black_reply = {}  



In [11]:
import sys 
version = 0.1
protocol_version = 2

class GTPInterface2:
  """
Interface permettant d'utiliser le protocole GTP pour contrôler le programme. 
Chaque commande GTP implémentée renvoie une réponse sous forme de chaîne à l'utilisateur, ainsi qu'un booléen indiquant le succès ou l'échec de l'exécution de la commande. 
L'interface contient un agent qui décide des mouvements à effectuer sur demande, ainsi qu'un gamestate qui contient l'état actuel du jeu.
  """
  AGENTS = {"decisive_move": DecisiveMoveMctsagent, "lgr": LGRMctsagent,"dca": DCAMctsagent, "basic": RaveMctsagent,"poolrave": PoolraveMctsagent} 
  
  def __init__(self, agent_name="basic"):
    """Initialise la liste des commandes disponibles, en liant les noms appropriés aux fonctions définies dans ce fichier. """
    commands={}
    commands["name"] = self.gtp_name
    commands["version"] = self.gtp_version
    commands["protocol_version"] = self.gtp_protocol
    commands["known_command"] = self.gtp_known
    commands["list_commands"] = self.gtp_list
    commands["quit"] = self.gtp_quit
    commands["boardsize"] = self.gtp_boardsize
    commands["size"] = self.gtp_boardsize
    commands["clear_board"] = self.gtp_clear
    commands["play"] = self.gtp_play
    commands["genmove"] = self.gtp_genmove
    commands["showboard"] = self.gtp_show
    commands["print"] = self.gtp_show
    commands["set_time"] = self.gtp_time
    commands["winner"] = self.gtp_winner
    commands["hexgui-analyze_commands"] = self.gtp_analyze
    commands["agent"] = self.gtp_agent
    self.commands = commands
    self.game = Gamestate(8)
    self.agent_name = agent_name
    try:
      self.agent = self.AGENTS[agent_name]()
    except KeyError:
      print("Unknown agent defaulting to basic")
      self.agent_name = "basic"
      self.agent = self.AGENTS[agent_name]()
    self.agent.set_gamestate(self.game)
    self.move_time = 10
    self.last_move = None

  def send_command(self, command):
    """Analyse la commande donnée en un nom de fonction et des arguments, l'exécute et renvoie la réponse.."""
    parsed_command = command.split()
    #first word specifies function to call, the rest are args
    name = parsed_command[0]
    args = parsed_command[1:]
    if(name in self.commands):
      return self.commands[name](args)
    else:
      return (False, "Unrecognized command")
  def register_command(self, name, command):
    """ Ajoute une nouvelle commande à la liste des commandes sous le nom qui appelle la fonction commande, écrasera également les anciennes commandes."""
    self.commands[name] = command
  def gtp_name(self, args):
    """ retourne le nom du programme """
    return (True, "RHex")
  def gtp_version(self, args):
    """Retourne la version du programme"""
    return (True, str(version))
  def gtp_protocol(self, args):
    """Retourne la  version GTP utilisé"""
    return(True, str(protocol_version))
  def gtp_known(self, args):
    """Retourne un booléen indiquant si le nom de la commande passée est une commande connue."""
    if(len(args)<1):
      return (False, "Not enough arguments")
    if(args[0] in self.commands):
      return (True, "true")
    else:
      return (True, "false")

  def gtp_list(self, args):
    """Retourne une liste de tous les noms de commandes connus."""
    ret=''
    for command in self.commands:
      ret+='\n'+command
    return (True, ret)

  def gtp_quit(self, args):
    """Exit le programme."""
    sys.exit()
  def gtp_boardsize(self, args):
    """Définit la taille du board game (efface également le board)."""
    if(len(args)<1):
      return (False, "Not enough arguments")
    try:
      size = int(args[0])
    except ValueError:
      return (False, "Argument is not a valid size")
    if size<1:
      return (False, "Argument is not a valid size")
    
    self.game = Gamestate(size)
    self.agent.set_gamestate(self.game)
    self.last_move = None
    return (True, "")

  def gtp_clear(self, args):
    """
    Clear the game board.
    """
    self.game = Gamestate(self.game.size)
    self.agent.set_gamestate(self.game)
    self.last_move = None
    return (True, "")

  def gtp_play(self, args):
    """ Jouer  a stone d 'une couleur ou cellule donnée
    1st arg = colour (white/w or black/b)
    2nd arg = cell (i.e. g5)
    Remarque : l'ordre de jeu n'est pas imposé, mais les tours non ordonnés entraîneront la réinitialisation de l'arbre de recherche. """
    if(len(args)<2):
      return (False, "Not enough arguments")
    try:
      x = ord(args[1][0].lower())-ord('a')
      y = int(args[1][1:])-1
      if(x<0 or y<0 or x>=self.game.size or y>=self.game.size):
        return (False, "Cell out of bounds")
      if args[0][0].lower() == 'w':
        self.last_move = (x, y)
        if self.game.turn() == Gamestate.PLAYERS["white"]:
          self.game.play((x,y))
          self.agent.move((x,y))
          return (True, "")
        else:
          self.game.place_white((x,y))
          self.agent.set_gamestate(self.game)
          self.last_move = None
          return (True, "")

      elif args[0][0].lower() == 'b':
        self.last_move = (x, y)
        if self.game.turn() == Gamestate.PLAYERS["black"]:
          self.game.play((x,y))
          self.agent.move((x,y))
          return (True, "")
        else:
          self.game.place_black((x,y))
          self.agent.set_gamestate(self.game)
          self.last_move = None
          return (True, "")
      else:
        return(False, "Player not recognized")
    except ValueError:
      return (False, "Malformed arguments")

  def gtp_genmove(self, args):
    """Permettre à l'agent de jouer une pierre de la couleur donnée (white/w or black/b)
       Remarque : l'ordre de jeu n'est pas imposé, mais les tours non ordonnés entraîneront la réinitialisation de l'arbre de recherche des agents."""
    #si l'utilisateur spécifie un joueur, génère le mouvement approprié sinon il continue avec le tour actuel.
    if(len(args)>0):
      if args[0][0].lower() == 'w':
        if self.game.turn() != Gamestate.PLAYERS["white"]:
          self.game.set_turn(Gamestate.PLAYERS["white"])
          self.agent.set_gamestate(self.game)
          self.last_move = None
      elif args[0][0].lower() == 'b':
        if self.game.turn() != Gamestate.PLAYERS["black"]:
          self.game.set_turn(Gamestate.PLAYERS["black"])
          self.agent.set_gamestate(self.game)
          self.last_move = None
      else:
        return (False, "Player not recognized")

    move = self.agent.special_case(self.last_move)
    self.agent.search(self.move_time)

    if not move:
      move = self.agent.best_move()

    if(move == Gamestate.GAMEOVER):
      return (False, "The game is already over")
    self.game.play(move)
    self.agent.move(move)
    self.last_move = move
    return (True, chr(ord('a')+move[0])+str(move[1]+1))

  def gtp_time(self, args):
    """Modifier le temps par déplacement alloué à l'agent de recherche (en unités de secondes)"""
    if(len(args)<1):
      return (False, "Not enough arguments")
    try:
      time = int(args[0])
    except ValueError:
      return (False, "Argument is not a valid time limit")
    if time<1:
      return (False, "Argument is not a valid time limit")
    self.move_time = time
    return (True, "")
  def gtp_show(self, args):
    """ Renvoie une représentation ascii de l'état actuel du game board"""
    if(len(args)<1):
      return (True, str(self.game))
    elif args[0][0].lower() == 'w':
      return (True, self.game.show_board(Gamestate.PLAYERS["white"]))
    elif args[0][0].lower() == 'b':
      return (True, self.game.show_board(Gamestate.PLAYERS["black"]))
    else:
      return (False, "Player not recognized")    
  def gtp_winner(self, args):
    """Retourne le gagnant de la partie en cours (black or white), aucun si indécis. """
    if(self.game.winner()==Gamestate.PLAYERS["white"]):
      return (True, "white")
    elif(self.game.winner()==Gamestate.PLAYERS["black"]):
      return (True, "black")
    else:
      return (True, "none")

  def gtp_analyze(self, args):
    """Ajouté pour éviter le plantage avec le gui mais pas encore implémenté."""
    return (True, "")
  def gtp_agent(self, args):
    """Changez l'agent utilisé par le joueur parmi les options disponibles."""

    if len(args)<1:
      ret="Available agents:"
      for agent in self.AGENTS.keys():
        if self.agent_name == agent:
          ret+="\n\033[92m"+agent+"\033[0m"
        else:
          ret+="\n"+agent
      return (True, ret)
    else:
      try:
        self.agent = self.AGENTS[args[0]](self.game)
      except KeyError:
        return (False, "Unknown agent")
      self.agent_name = args[0]
      return (True, "")

In [12]:

#code pour tester différents joueurs de reverse hex les uns contre les autres
basic = GTPInterface2("basic")
improved = GTPInterface2("dca")
black = basic
white = improved
score1 = 0
score2 = 0
basic.gtp_boardsize("9")
basic.gtp_time("1")
improved.gtp_boardsize("9")
improved.gtp_time("1")

for i in range(20):
  black.gtp_clear("")
  white.gtp_clear("")

  while black.gtp_winner("")[1] == "none":
    move = white.gtp_genmove("")[1]
    black.gtp_play(["white ", move])
    if black.gtp_winner("")[1] == "none":
      move = black.gtp_genmove("")[1]
      white.gtp_play(["b ", move])
        
  if improved == black and black.gtp_winner("")[1] == "black":
    score1 += 1
  elif improved == white and white.gtp_winner("")[1] == "white":
    score2 += 1
  print ("score")
  print(black.gtp_winner("")[1])
  print (score1)
  print (score2)
  print (i)
  print (black.gtp_show("")[1])
  black, white = white, black

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:95: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:97: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


score
black
0
0
0

  A  B  C  D  E  F  G  H  I  
1  O  @  O  O  @  O  @  @  O  O
 2  O  @  O  O  O  @  O  O  @  O
  3  @  O  @  @  @  @  @  O  @  O
   4  @  @  @  O  @  O  O  O  @  O
    5  O  O  O  @  O  O  @  @  @  O
     6  @  @  O  O  @  O  O  O  @  O
      7  @  O  O  O  @  @  O  @  O  O
       8  @  @  O  @  @  O  @  O  O  O
        9  @  O  O  O  @  @  @  O  O  O
            @  @  @  @  @  @  @  @  @  
score
black
1
0
1

  A  B  C  D  E  F  G  H  I  
1  O  @  @  @  O  @  O  O  @  O
 2  @  O  O  @  O  @  O  O  O  O
  3  O  @  @  @  O  @  O  @  @  O
   4  O  O  O  O  O  O  @  O  @  O
    5  @  @  O  O  @  @  O  O  O  O
     6  O  @  @  O  @  @  O  O  O  O
      7  O  O  @  O  @  O  @  O  @  O
       8  O  @  @  @  O  @  @  O  O  O
        9  @  @  O  O  @  @  @  @  @  O
            @  @  @  @  @  @  @  @  @  
score
white
1
1
2

  A  B  C  D  E  F  G  H  I  
1  O  @  @  @  O  O  O  @  O  O
 2  O  O  @  @  O  @  O  O  O  O
  3  O  O  @  @  @  @  O  @  @  O
   4  O  O  .  @  O  @  @ 

In [15]:
# Un test pour vérifier que le lecteur d'analyse des cellules mortes détecte correctement les cellules mortes.

basic = GTPInterface2("basic")
improved = GTPInterface2("dca")
black = basic
white = improved
score1 = 0
score2 = 0
basic.gtp_boardsize(["7"])
basic.gtp_time("1")
improved.gtp_boardsize(["7"])
improved.gtp_time("1")
improved.gtp_play(["b", "d2"])
improved.gtp_play(["b", "c2"])
improved.gtp_play(["b", "b3"])
improved.gtp_play(["b", "b4"])
improved.gtp_play(["b", "c4"])
improved.gtp_play(["b", "d3"])
print(improved.gtp_genmove("")[1])
print(improved.gtp_genmove("w")[1])
print(improved.gtp_show("")[1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:95: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:97: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


a4
c3

  A  B  C  D  E  F  G  
1  .  .  .  .  .  .  .  O
 2  .  .  @  @  .  .  .  O
  3  .  @  O  @  .  .  .  O
   4  O  @  @  .  .  .  .  O
    5  .  .  .  .  .  .  .  O
     6  .  .  .  .  .  .  .  O
      7  .  .  .  .  .  .  .  O
          @  @  @  @  @  @  @  


In [ ]:
def main():
  """Main function, envoie simplement l'entrée de l'utilisateur sur l'interface gtp et imprime les réponses."""
  
  interface = GTPInterface2("dca")
  while True:
    command = input()
    success, response = interface.send_command(command)
    print(("= " if success else "? ")+response+'\n')

if __name__ == "__main__":
  main()